In [2]:
from vqa import *

In [50]:
train_img_dir = r"C:\Users\ved67\UIUC\CS521\CS521_HWs\hw5\VQA_Train\scene_img_abstract_v002_train2017"
train_img_prefix = "abstract_v002_train2015_"
train_question_json = r"C:\Users\ved67\UIUC\CS521\CS521_HWs\hw5\VQA_Train\OpenEnded_abstract_v002_train2017_questions.json"
train_answer_json = r"C:\Users\ved67\UIUC\CS521\CS521_HWs\hw5\VQA_Train\abstract_v002_train2017_annotations.json"

valid_img_dir = r"C:\Users\ved67\UIUC\CS521\CS521_HWs\hw5\VQA_Valid\scene_img_abstract_v002_val2017"
valid_img_prefix = "abstract_v002_val2015_"
valid_question_json = r"C:\Users\ved67\UIUC\CS521\CS521_HWs\hw5\VQA_Valid\OpenEnded_abstract_v002_val2017_questions.json"
valid_answer_json = r"C:\Users\ved67\UIUC\CS521\CS521_HWs\hw5\VQA_Valid\abstract_v002_val2017_annotations.json"

transform = transforms.Compose([
    transforms.Resize((224,224))
])
tokenizer = SimpleTokenizer(max_len=50)
valid_dataset = VQADataset(valid_img_dir, valid_img_prefix, valid_question_json, valid_answer_json, transform, tokenizer)
complementary_pairs = []
for img_id in valid_dataset.img_ids:
    if len(str(img_id)) < 9:
        complementary_id = int("900"+str(img_id)+"0")
        if complementary_id in valid_dataset.img_ids:
            img1_data = {}
            img2_data = {}
            image, question_tokens, answer_tokens = valid_dataset[valid_dataset.img_ids.index(img_id)]
            img1_data["image"] = image
            img1_data["question_tokens"] = question_tokens
            img1_data["answer_tokens"] = answer_tokens
            image, question_tokens, answer_tokens = valid_dataset[valid_dataset.img_ids.index(complementary_id)]
            img2_data["image"] = image
            img2_data["question_tokens"] = question_tokens
            img2_data["answer_tokens"] = answer_tokens
            if str(img1_data["question_tokens"].tolist()) != str(img2_data["question_tokens"].tolist()):
                continue
            if len(tokenizer.decode(img1_data['question_tokens'])) > 40:
                continue
            print(complementary_id, img_id)
            # print(img1_data["question_tokens"], img2_data["question_tokens"])
            # print("cont")
            # print(img1_data["answer_tokens"], img2_data["answer_tokens"])
            # print("end")
            complementary_pairs.append((img1_data, img2_data))
    if len(complementary_pairs) >= 10:
        break
len(complementary_pairs)

900200090 20009
900200220 20022
900200280 20028
900200340 20034
900200380 20038
900200450 20045
900200470 20047
900200500 20050
900200520 20052
900200540 20054


10

In [ ]:
os.environ["API_KEY"] = 'API_KEY'

In [52]:
import google.generativeai as genai
model_name = 'gemini-1.5-flash'
genai.configure(api_key=os.environ["API_KEY"])
model = genai.GenerativeModel(model_name)

def vqa(model, prompt, image):
    response = model.generate_content([prompt, image])
    return response.text

In [53]:
prompt_template = """
Given the image and the question, answer the question.

The answer should be in the format:

Explanation: <the explanation of the answer>
#Answer: <yes/no>

query: {question}
"""

def check_answer(model_answer, answer):
    model_answer = model_answer.strip().split('#Answer:')[1].lower().strip()
    if 'n' in model_answer and 'n' in answer:
        return True, model_answer
    elif 'y' in model_answer and 'y' in answer:
        return True, model_answer
    else:
        assert model_answer == 'no' or model_answer == 'yes', f"Invalid model answer: {model_answer}"
        return False, model_answer

In [61]:
import pandas as pd

def image_vqa(model, img_data):
    vqa_data = {}
    image, question_tokens, answer_tokens = img_data['image'], img_data['question_tokens'], img_data['answer_tokens']
    question = tokenizer.decode(question_tokens)
    answer = tokenizer.decode(answer_tokens)
    prompt = prompt_template.format(question=question)
    model_answer = vqa(model, prompt, image)
    evaluation, shortened_answer = check_answer(model_answer, answer)
    vqa_data['question'] = question
    vqa_data['answer'] = answer
    vqa_data['model_answer'] = model_answer
    vqa_data['evaluation'] = evaluation
    vqa_data['shortened_answer'] = shortened_answer
    vqa_data['image'] = image
    return vqa_data

model_performance_data = {}
for i in range(10):
    img1_data, img2_data = complementary_pairs[i]
    image, question_tokens, answer_tokens = valid_dataset[i]
    question = tokenizer.decode(question_tokens)
    
    model_performance_data[i] = (image_vqa(model, img1_data), image_vqa(model, img2_data))

In [62]:
import matplotlib.pyplot as plt
import numpy as np

def show_pairs(img1_data, img2_data):
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    ax[0].imshow(np.array(img1_data['image']))
    ax[0].axis('off')
    ax[0].set_title('Image 1')
    ax[1].imshow(np.array(img2_data['image']))
    ax[1].axis('off')
    ax[1].set_title('Image 2')
    plt.suptitle(f'Question: {img1_data["question"]}')
    print(f'Image 1 answer: {img1_data["answer"]}, {img1_data["shortened_answer"]}')
    print(f'Image 2 answer: {img2_data["answer"]}, {img2_data["shortened_answer"]}')
    plt.show()

In [64]:
pair_performance_data = {}
normal_correct = 0
normal_total = 0
pair_correct = 0
for i, (img1_data, img2_data) in model_performance_data.items():
    pair_performance_data[i] = {}
    pair_performance_data[i]['question1'] = img1_data['question']
    pair_performance_data[i]['answer1'] = img1_data['answer']
    pair_performance_data[i]['model_answer1'] = img1_data['model_answer']
    pair_performance_data[i]['evaluation1'] = img1_data['evaluation']
    pair_performance_data[i]['shortened_answer1'] = img1_data['shortened_answer']
    pair_performance_data[i]['question2'] = img2_data['question']
    pair_performance_data[i]['answer2'] = img2_data['answer']
    pair_performance_data[i]['model_answer2'] = img2_data['model_answer']
    pair_performance_data[i]['evaluation2'] = img2_data['evaluation']
    pair_performance_data[i]['shortened_answer2'] = img2_data['shortened_answer']
    pair_performance_data[i]['evaluation'] = img1_data['evaluation'] and img2_data['evaluation']
    normal_correct += img1_data['evaluation']
    normal_total += 1
    normal_correct += img2_data['evaluation']
    normal_total += 1
    pair_correct += img1_data['evaluation'] and img2_data['evaluation']

print(f"Normal accuracy: {normal_correct/normal_total}")
print(f"Pair accuracy: {pair_correct/(normal_total//2)}")

pair_performance_df = pd.DataFrame.from_dict(pair_performance_data, orient='index')
pair_performance_df.head()

Normal accuracy: 0.6
Pair accuracy: 0.4


,question1,answer1,model_answer1,evaluation1,shortened_answer1,question2,answer2,model_answer2,evaluation2,shortened_answer2,evaluation
0,is the young boy in love with the dog?,yes,Explanation: The image shows a young boy sitti...,False,no,is the young boy in love with the dog?,no,Explanation:The image shows a young boy in a s...,True,no,False
1,is he camping?,yes,Explanation:The image shows a man sitting on l...,True,yes,is he camping?,no,Explanation:The image shows a man carrying log...,True,no,True
2,is the girl lonely?,yes,Explanation: The image shows a girl sitting on...,False,no,is the girl lonely?,no,Explanation: The image shows a girl standing n...,True,no,False
3,is he happy?,yes,Explanation: The image shows a boy playing on ...,True,yes,is he happy?,yes,Explanation: The image shows a boy actively pl...,True,yes,True
4,are there clouds in the sky?,no,Explanation:The image shows a clear sky with a...,True,no,are there clouds in the sky?,yes,"Explanation:The image shows a single, small cl...",True,yes,True


In [66]:
pair_performance_df.to_excel('model_performance.xlsx', index=False)